In [1]:
import tensorflow as tf
import numpy as np
import os
from skimage import io
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

def load_and_split_image(image_path, img_size=(192, 288), debug=False):
    # Load full image (380, 1140, 3)
    full_image = io.imread(image_path)  
    
    # Ensure image is the expected shape
    if full_image.shape[1] < 1140:
        print(f"❌ Error: Image width too small - {image_path}, Shape: {full_image.shape}")
        return None, None

    # Split into B-scan (left) and mask (right)
    bscan = full_image[:, :570, :]  # Left half: OCT B-scan
    mask = full_image[:, 570:, :]   # Right half: Segmentation mask

    if debug:
        print("Mask shape before resizing:", mask.shape)
        unique_colors_before = np.unique(mask.reshape(-1, mask.shape[2]), axis=0)
        print("Unique colors before resizing:\n", unique_colors_before)

    # Resize to match U-Net input shape
    # -- Keep bilinear for bscan
    bscan = tf.image.resize(
        bscan,
        img_size,
        method=tf.image.ResizeMethod.BILINEAR
    )
    
    # -- Use nearest-neighbor for mask
    mask = tf.image.resize(
        mask,
        img_size,
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )
    mask = tf.cast(mask, tf.uint8)  # Force back to uint8
    mask = mask.numpy()  # Convert to NumPy array

    #if debug:
        #print("Mask shape after resizing:", mask.shape)
        #unique_colors_after = np.unique(mask.reshape(-1, mask.shape[2]), axis=0)
        #print("Unique colors after resizing:\n", unique_colors_after)

    return bscan.numpy(), mask



In [2]:
# Define color-to-class mapping (RGB values for each class)
COLOR_MAP = {
    0: (255, 0, 0),       # Red: SRF
    1: (255, 192, 203),   # Pink: IS/OS
    2: (0, 0, 255),       # Blue: PED
    3: (0, 255, 0),       # Green: IRF
    4: (255, 255, 0),     # Yellow: SHRM
    5: (0, 0, 0)          # Black: Background
}

def rgb_to_label(mask_rgb, tolerance=5):
    """
    Convert RGB mask to one-hot encoded labels while handling slight color shifts.
    """
    H, W, _ = mask_rgb.shape
    label_mask = np.zeros((H, W, len(COLOR_MAP)), dtype=np.uint8)

    for class_idx, rgb in COLOR_MAP.items():
        rgb = np.array(rgb)  # Convert to NumPy array
        # Allow small variations in color matching
        matches = np.all(np.abs(mask_rgb - rgb) <= tolerance, axis=-1)
        label_mask[matches, class_idx] = 1  # One-hot encoding

    return label_mask



In [3]:
def load_dataset_from_txt(txt_path, base_dir="/kaggle/input/amd-sd/AMD-SD/images", img_size=(192, 288)):
    with open(txt_path, "r") as f:
        filenames = [line.strip() for line in f.readlines()]
    
    images = []
    masks = []
    for i, filename in enumerate(filenames):
        # Extract eye ID from filename (e.g., "1_1.png" → eye_id = "1")
        eye_id = filename.split("_")[0]
        image_path = os.path.join(base_dir, eye_id, filename)

        #debug_flag = (i < 3)
        # Load and split image (with debug flag for the first 3 images)
        bscan, mask_rgb = load_and_split_image(image_path, img_size, #debug=debug_flag)
        
        # Convert mask to class labels
        mask_label = rgb_to_label(mask_rgb)
        
        images.append(bscan)
        masks.append(mask_label)
    
    return np.array(images), np.array(masks)

# Load training and validation data
X_train, Y_train = load_dataset_from_txt("/kaggle/input/amd-sd/AMD-SD/training.txt")
X_val, Y_val = load_dataset_from_txt("/kaggle/input/amd-sd/AMD-SD/validation.txt")

# Normalize images to [0, 1]
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0

Y_train = np.argmax(Y_train, axis=-1)  # Convert one-hot to class indices (shape: (None, 192, 288))
Y_val = np.argmax(Y_val, axis=-1)


NameError: name 'debug_flag' is not defined

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

def UNetResnet(input_size=(192, 288, 3), num_classes=6):
    base_model = ResNet50(input_shape=input_size, include_top=False, weights="imagenet")

    # Extract relevant layers for skip connections
    skip_connections = [
        base_model.get_layer("conv1_relu").output,   # (96, 144, 64)
        base_model.get_layer("conv2_block3_out").output,  # (48, 72, 256)
        base_model.get_layer("conv3_block4_out").output,  # (24, 36, 512)
        base_model.get_layer("conv4_block6_out").output,  # (12, 18, 1024)
    ]
    
    encoder_output = base_model.get_layer("conv5_block3_out").output  # (6, 9, 2048)
    
    # Decoder: Upsample step by step
    x = layers.Conv2DTranspose(1024, (3,3), strides=(2,2), padding="same", activation="relu")(encoder_output)  # (12, 18, 1024)
    x = layers.Concatenate()([x, skip_connections[-1]])

    x = layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding="same", activation="relu")(x)  # (24, 36, 512)
    x = layers.Concatenate()([x, skip_connections[-2]])

    x = layers.Conv2DTranspose(256, (3,3), strides=(2,2), padding="same", activation="relu")(x)  # (48, 72, 256)
    x = layers.Concatenate()([x, skip_connections[-3]])

    x = layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding="same", activation="relu")(x)  # (96, 144, 128)
    x = layers.Concatenate()([x, skip_connections[-4]])

    x = layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding="same", activation="relu")(x)  # (192, 288, 64)

    # Final segmentation layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation="softmax", padding="same")(x)  # (192, 288, num_classes)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# Instantiate and compile the model
model_resnet_unet = UNetResnet()
optimizer = Adam(learning_rate=0.0001)
model_resnet_unet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model
history_resnet_unet = model_resnet_unet.fit(
    X_train, 
    Y_train, 
    epochs=10, 
    batch_size=8, 
    validation_data=(X_val, Y_val)
)

# Evaluate the model
val_loss, val_accuracy = model_resnet_unet.evaluate(X_val, Y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
model_resnet_unet.save('model.h5')


In [ ]:
COLOR_MAP = {
    0: (255, 0, 0),       # Red: SRF
    1: (255, 192, 203),   # Pink: IS/OS
    2: (0, 0, 255),       # Blue: PED
    3: (0, 255, 0),       # Green: IRF
    4: (255, 255, 0),     # Yellow: SHRM
    5: (0, 0, 0)          # Black: Background
}

def rgb_to_label(mask_rgb, tolerance=5):
    """
    Convert RGB mask to one-hot encoded labels while handling slight color shifts.
    """
    H, W, _ = mask_rgb.shape
    label_mask = np.zeros((H, W, len(COLOR_MAP)), dtype=np.uint8)

    for class_idx, rgb in COLOR_MAP.items():
        rgb = np.array(rgb)  # Convert to NumPy array
        # Allow small variations in color matching
        matches = np.all(np.abs(mask_rgb - rgb) <= tolerance, axis=-1)
        label_mask[matches, class_idx] = 1  # One-hot encoding

    return label_mask

def label_to_rgb(mask):
    """Convert multi-class labels to RGB color mapping."""
    H, W = mask.shape
    rgb_mask = np.zeros((H, W, 3), dtype=np.uint8)

    for class_idx, rgb in COLOR_MAP.items():
        class_mask = (mask == class_idx)
        rgb_mask[class_mask] = rgb

    return rgb_mask


# Select a random index from validation set
idx = np.random.randint(0, X_val.shape[0])

# Get the image and ground truth mask
image = X_val[2]  # Input image
true_mask = Y_val[2]  # True segmentation mask

# Predict the mask using the trained model
predicted_mask = model_resnet_unet.predict(np.expand_dims(image, axis=0))  # Add batch dim

# Convert model prediction to class indices
predicted_mask = np.argmax(predicted_mask, axis=-1)[0]  # Remove batch dim

# Convert masks to RGB
true_mask_rgb = label_to_rgb(true_mask)
predicted_mask_rgb = label_to_rgb(predicted_mask)

# Plot the original image, ground truth mask, and predicted mask
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Original Image
ax[0].imshow(image)
ax[0].set_title("Original Image")
ax[0].axis("off")

# Ground Truth Mask
ax[1].imshow(true_mask_rgb)
ax[1].set_title("Ground Truth Mask")
ax[1].axis("off")

# Predicted Mask
ax[2].imshow(predicted_mask_rgb)
ax[2].set_title("Predicted Mask")
ax[2].axis("off")

plt.show()
